# Session 2.5: Vektordatenbanken

## Quellen

- source: https://milvus.io/docs/de/v2.4.x/build_RAG_with_milvus_and_ollama.md
    - milvus lite: https://github.com/milvus-io/milvus-lite
    

## Praxis

### 🛠️ Imports

In [ ]:
from aiworkshop_utils.standardlib_imports import os, json, base64, logging, Optional, Any, Dict, List, pprint, shutil, glob
from aiworkshop_utils.thirdparty_imports import AutoTokenizer, load_dotenv, requests, BaseModel, Field, pd, cosine_similarity, plt, np, DataType, MilvusClient, tqdm, SentenceTransformer
from aiworkshop_utils.custom_utils import show_pretty_json, encode_image, load_md_sections_from_path
from aiworkshop_utils.jupyter_imports import Markdown, HTML, widgets, display, JSON
from aiworkshop_utils.docling_imports import PyPdfiumDocumentBackend, HybridChunker, InputFormat, AcceleratorDevice, AcceleratorOptions, PdfPipelineOptions, DocumentConverter, PdfFormatOption, WordFormatOption, SimplePipeline
from aiworkshop_utils.openai_imports import openaisdk_client_chat_completions_create
from aiworkshop_utils import config
from aiworkshop_utils import embedders

### 💡 Konzepte: Vektordatenbanken und RAG (Retrieval Augmented Generation)

- milvus
    - Info: für kleinere Datensätze mit bis zu einigen Millionen Vektoren -> Milvus Lite ist ok
    - für Datensätze bis zu 100 Millionen Vektoren -> besser Milvus Standalone

- ollama
    - Info: oft vorteilhaft, spezialisierte Modelle für Embedding-Aufgaben einzusetzen
    - kann Leistung in Aufgaben wie semantischer Suche verbessern, sind auch ressourcenschonender, effizienter

- Muss das Modell, das Embeddings vornimmt, das gleiche sein, wie das Generierungsmodell einer Chat-Anwendung?
    - Das Embedding Modell muss nicht das Generierungs-Modell sein, das später benützt wird bzw. muss es nicht von der gleichen Modell-Familie sein in RAG Prozesen -> Solange das gleiche Embedding Modell für alle Embeddings benutzt wird, ist alles gut
        - Prompt wird mit Embedding Modell eingebettet, dann findet eine semantische Suche in einer Vektordatenbank statt, Chunks der eingebetteten Dokumente kommen zurück
        - Generierungsmodell benützt den Prompt und die gefundenen Chunks und gibt dies in den Kontext -> es gibt hier keine Überschneidug mit dem Embedding-Vektorraum
        - Quelle: https://ericmjl.github.io/blog/2024/1/15/your-embedding-model-can-be-different-from-your-text-generation-model/

### ⚡ Milvus Beispiel 1

In [ ]:
# -------------------------------
# 1. Verbindung zu Milvus Lite herstellen und Collection erstellen
# -------------------------------

# Verbindung zum Milvus-Server herstellen
milvus_client = MilvusClient(config.MILVUS_DB)

# Collection-Name und Vektordimension definieren
collection_name = config.MILVUS_COLLECTION_A
embedding_dims = 768 # NOMIC-Model Default

# Überprüfen, ob die Collection bereits existiert, und ggf. löschen
if milvus_client.has_collection(collection_name=collection_name):
    milvus_client.drop_collection(collection_name=collection_name)

# Collection erstellen
milvus_client.create_collection(
    collection_name=collection_name,
    dimension=embedding_dims,
    primary_field_name='id',
    id_type=DataType.INT64,
    vector_field_name='vector',
    metric_type='IP',  # Inner Product Distance
    auto_id=True, # erstellt automatisch IDs
    consistency_level='Strong',
)

print(f"Collection '{collection_name}' mit Dimension {embedding_dims} erstellt.")

In [ ]:
# -------------------------------
# 2. Beispieltexte definieren und Embeddings erstellen
# -------------------------------

# Beispiel-Dokumente
docs = [
    "Artificial intelligence was founded as an academic discipline in 1956.",
    "Alan Turing was the first person to conduct substantial research in AI.",
    "Born in Maida Vale, London, Turing was raised in southern England.",
    "In 2022, OpenAI made ChatGPT publicly available.",
]
subject = "history"

milvusbsp1_embeddings = embedder_ollama.get_embeddings(docs)

print(f"Embeddings für alle Dokumente abgerufen. Anzahl: {len(milvusbsp1_embeddings)}")
show_pretty_json(milvusbsp1_embeddings)

In [ ]:
# -------------------------------
# 3. Daten in Milvus Lite einfügen und Suche durchführen
# -------------------------------

# Daten vorbereiten – jeder Eintrag enthält eine eindeutige ID, den Vektor, den Text und ein zusätzliches Feld "subject"
data = [
    {"id": i, "vector": milvusbsp1_embeddings[i], "text": docs[i], "subject": subject}
    for i in range(len(docs))
]

insert_res = milvus_client.insert(collection_name=collection_name, data=data)
print("Insert result:", insert_res)

In [ ]:
# -------------------------------
# 4. Suche: Verwende z.B. das Embedding des ersten Dokuments als Suchvektor
# -------------------------------
query_vector = milvusbsp1_embeddings[0]
search_res = milvus_client.search(
    collection_name=collection_name,
    data=[query_vector],
    filter="subject == 'history'",
    limit=2,
    output_fields=["text", "subject"]
)

print("Search result:")
show_pretty_json(search_res[0])

In [ ]:
# -------------------------------
# 5. Beispiel-Prompt an das Conversational Model (llama3.1) senden
# -------------------------------
# Baue einen Prompt mit den Suchergebnissen als Kontext.
# Hier werden alle gefundenen Texte zusammengeführt.
# Für eine einzelne Abfrage: Greife auf die Trefferliste der ersten Query zu.
hits = search_res[0]  # Direkt verwenden, da hits bereits eine Liste von Dictionaries ist
context_texts = " ".join([hit["entity"]["text"] for hit in hits])

print("Context texts:")
show_pretty_json(context_texts)

conversation_prompt = (
    f"Basierend auf den folgenden Informationen: {context_texts}\n"
    "Benutze nur diese Informationen, um die Frage zu beantworten.\n"
    "Beantworte: Was sind wichtige Meilensteine in der Geschichte der künstlichen Intelligenz?"
)

print("Conversation prompt:")
show_pretty_json(conversation_prompt)

# Sende den Request an den Conversational-Endpoint
conv_response = requests.post(
    config.OAPI_GENERATE_URL,
    json={"model": config.OMODEL_MISTRALNEMO, "prompt": conversation_prompt}
)

# Lies den kompletten rohen Response-Text aus und teile ihn in einzelne Zeilen
raw_response = conv_response.text.strip()
lines = raw_response.splitlines()

# Fasse die 'response'-Teile aus jeder Zeile zusammen
full_response = ""
for line in lines:
    try:
        json_obj = json.loads(line)
        full_response += json_obj.get("response", "")
    except json.JSONDecodeError as e:
        print("Fehler beim Parsen der Zeile:", line, e)

print("Full conversation model output:")
print(full_response)

In [ ]:
# Überprüfen, ob die Collection bereits existiert, und ggf. löschen
if milvus_client.has_collection(collection_name=collection_name):
    milvus_client.drop_collection(collection_name=collection_name)

### ⚡ Milvus Beispiel 2

In [ ]:
# -------------------------------
# 1. Verbindung zu Milvus Lite herstellen und Collection erstellen
# -------------------------------

# Verbindung zum Milvus-Server herstellen
milvus_client = MilvusClient(config.MILVUS_DB)

# Collection-Name und Vektordimension definieren
collection_name = config.MILVUS_COLLECTION_A
embedding_dims = 768 # NOMIC-Model Default

# Überprüfen, ob die Collection bereits existiert, und ggf. löschen
if milvus_client.has_collection(collection_name=collection_name):
    milvus_client.drop_collection(collection_name=collection_name)

# Collection erstellen
milvus_client.create_collection(
    collection_name=collection_name,
    dimension=embedding_dims,
    primary_field_name='id',
    id_type=DataType.INT64,
    vector_field_name='vector',
    metric_type='IP',  # Inner Product Distance
    auto_id=True, # erstellt automatisch IDs
    consistency_level='Strong',
)

print(f"Collection '{collection_name}' mit Dimension {embedding_dims} erstellt.")

In [ ]:
# -------------------------------
# 2. Beispieltexte definieren und Embeddings erstellen
# -------------------------------

example_text_lines = load_md_sections_from_path("milvus_docs/en/faq/*.md") # Chunking eines MD-Files!
show_pretty_json(example_text_lines)

data = []
for i, line in enumerate(tqdm(example_text_lines, desc="Creating embeddings")):
    data.append(
        {
            "id": i, 
            "vector": embedder_ollama.get_embeddings(line), 
            "text": line
            }
            )

print(len(data))
show_pretty_json(data[0])

In [ ]:
# -------------------------------
# 3. A) Daten in Milvus Lite einfügen
# -------------------------------
milvus_client.insert(
    collection_name=config.MILVUS_COLLECTION_A, 
    data=data
)

In [ ]:
stats = milvus_client.get_collection_stats(config.MILVUS_COLLECTION_A)
print(stats)
print(stats['row_count']*embedding_dims)

In [ ]:
# === Disk Space Information ===
# Update this to match your Milvus data directory as configured in milvus.yaml
milvus_data_path = "milvus_db.db"
total, used, free = shutil.disk_usage(milvus_data_path)

print("Disk Space Info:")
print("  Total: {:.2f} GB".format(total / 2**30))
print("  Used:  {:.2f} GB".format(used / 2**30))
print("  Free:  {:.2f} GB".format(free / 2**30))
print("")

# === Milvus Collection Stats ===
stats = milvus_client.get_collection_stats(config.MILVUS_COLLECTION_A)
current_vectors = stats.get('row_count', 0)
print("Milvus Collection Stats:")
print("  Current number of vectors: {}".format(current_vectors))

# === Estimating Storage Usage per Vector ===
# Update embedding_dims to your actual vector dimension.
embedding_dims = 512  
# Assuming float32 (4 bytes per dimension)
bytes_per_vector = embedding_dims * 4  
print("Approximate storage used by one vector:")
print("  {} bytes ({:.2f} KB)".format(bytes_per_vector, bytes_per_vector / 1024))
print("")

# === Estimating Total Vector Storage Usage ===
total_vector_bytes = current_vectors * bytes_per_vector
print("Approximate storage used by vectors: {:.2f} MB".format(total_vector_bytes / (1024**2)))
print("")

# === Estimating Remaining Capacity ===
# Adjust max_vectors_estimate based on your deployment expectations.
max_vectors_estimate = 2_000_000  
remaining_vectors = max_vectors_estimate - current_vectors
print("Estimated remaining capacity (vector count): {:,}".format(remaining_vectors).replace(",", "."))

In [ ]:
# -------------------------------
# 3. B) Suche durchführen
# -------------------------------
question = "How is data stored in milvus?"

search_res = milvus_client.search(
    collection_name=config.MILVUS_COLLECTION_A,
    data=[embedder_ollama.get_embeddings(question)],  # embed the prompt!
    limit=3,  # Return top 3 results
    search_params={"metric_type": "IP", "params": {}},  # Inner product distance
    output_fields=["text"],  # Return the text field
)

In [ ]:
retrieved_lines_with_distances = [
    (res["entity"]["text"], res["distance"]) for res in search_res[0]
]

print(json.dumps(retrieved_lines_with_distances, indent=4))

In [ ]:
context = "\n".join(
    [line_with_distance[0] for line_with_distance in retrieved_lines_with_distances]
)

In [ ]:
print(context)

In [ ]:
# -------------------------------
# 5. Beispiel-Prompt an das Conversational Model (llama3.1) senden
# -------------------------------

SYSTEM_PROMPT = """
Human: You are an AI assistant. You are able to find answers to the questions from the contextual passage snippets provided.
"""
USER_PROMPT = f"""
Use the following pieces of information enclosed in <context> tags to provide an answer to the question enclosed in <question> tags.
<context>
{context}
</context>
<question>
{question}
</question>
"""

In [ ]:
# Format request as per Ollama API
data = {
    "model": config.OMODEL_MISTRALNEMO,
    "messages": [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_PROMPT},
    ],
    "stream": False
}

# Send the request
response = requests.post(config.OAPI_CHAT_URL, json=data)
response_json = response.json()

# Debugging: Print full response
print("Full API Response:", response_json)

# Extract message content
#print(response_json["message"]["content"])

In [ ]:
# Extract the assistant's message content
assistant_message = response_json.get("message", {}).get("content", "No response received.")

# Print it in a user-friendly format
print("\n🤖 **Assistant's Response:**\n")
print(assistant_message)

### 🏋️ **[ÜBUNG_2.5.01]** Docling-Milvus-Connection

- *Diese Übung ist Teil von ÜBUNG5. Speichere deine Ergebnisse und Notizen in einem File (Word), füge dann noch die restlichen Übungen dieser Session 2.4 hinzu und lade sie auf Moodle unter "Abgabe Übung 5" bis zum 20.05.25 hoch.*

- Bastle eine kleine Pipeline für [Parsing -> Chunking -> Embedding -> Indexing]
- Führe dies mit einem PDF-Dokument deiner Wahl durch
- Du kanst für Parsing und Chunking Docling benützen
- Für Embedding kannst du ein Ollama-Model verwenden
- Erstelle für Indexing eine Milvus Lite DB Instanz und füge dort die Vektoren der Chunks ein